In [2]:
# Check if python is 3.9.5
import arrow
import numpy as np
import os
import sys

from __init__ import PCAT

print(sys.version)
%load_ext autoreload
%autoreload 2
%pprint

3.9.5 (default, May 18 2021, 12:31:01) 
[Clang 10.0.0 ]
Pretty printing has been turned OFF


# Classifying the diversity of Land Surface (Sub)Models

Here, we try to explore and rank LSM for the help on deciding, which model to take. The criteria for it are listed in the notebook [02_submodel_criteria](./02_submodel_criteria.ipynb).

## Structure

For each model we should have a (very) short paragraph that describes the submodel. In addition rankings for all 7 criteria, as well as some reasoning for it should be provided. For each submodel we can then define a python dictionary and cncatenate all of them finally to save it as `csv` or `yaml` file.

In [3]:
all_models_ranked = []

---
---
## Models

Let $c_n \in (c_k)_{k \in \mathbb{N}}$ always be predefined constants and us start.

---
### CLM Submodels

We start with looking at the submodels of the CLM. Its [Technical Note](https://escomp.github.io/ctsm-docs/versions/master/html/tech_note/index.html) lists 30 submodules/submodels. These should be analyzed. To avoid a bias of ranking the models in a given order, we fetch them randomly:

<hr style="border-style: dashed;">

#### 22. External Nitrogen Cycle

The external nitrogen cycle can be thought of as processes around nitrogen that happen oustide of organisms, especially plants. They are important to describe long term ecosystem dynamics of nitrogen stocks.

The model parameters are:


<div style="width:100px">parameter</div> | <div style="width:100px">unit</div>  | <div style="width:300px">equation (relation)</div> | <div style="width:500px">description</div>  |
|:---:|:---:|:---:|:---|
$NF_{ndep\_sminn}$ | $g\ m^{-2}\ s^{-1}$ | - | total deposition of mineral nitrogen to land surface (NO$_{x}$, NH$_{y}$)
$CF_{ann\_ET}$ | $mm \ yr^{-1}$ | - | evapotranspiration rate
$NF_{nfix\_sminn}$ | $g\ m^{-2}\ s^{-1}$ | $c_1 (c_2 + CF_{ann\_ET}) / (c_3)$ (linear) | biological asymbiotic nitrogen fixation rate (BNF)
$k_{nitr}$ | $day^{-1}$ | - | maximum nitrification rate (set to 0.1)
$f_{nitr,p}$ | $day^{-1}$ | $[NH_{4}]\ k_{nitr}\ f(T)\ f(H_{2}0)\ f(pH)$ (linear?!) | potential nitrification rate
$frac_{anox}$ | - | $\exp{\text(multiple\ params)}$ (exponential) | anoxic fraction
$f_{denitr,p}$ | $day^{-1}$ | $min(f_{denitr,p,carbon\_lim}, f_{denitr,p,no3\_lim})\ frac_{anox}$ (linear?!) | potential denitrification rate (only anoxic soils)
$k_{1}$ | - | $max(c_1, c_2-c_3\ d_g)$ (linear) | function of gas diffusion
$f_{WFPS}$ | - | $max(c_1, c_2-c_3\ WFPS)$ (linear) | function of water filled pore space
$P_{NO_3:CO_2}$ | - | - |  CO$_2$ production/NO$_{3}$ concentration ratio
$P_{N_2:N_2O}$ | - | $max(0.16\ k_1, k_1\ \exp(-0.8\ P_{NO_3:CO_2}))\ f_{WPFS}$ (exponential) | nitrification/denitrification ratio
$WS_{tot\_soil}$ | $kg\ m^{2}$ | - | total mass of soil water over column
$NS_{sminn}$ | $g\ m^{-2} s^{-1}$ | - | soulable fraction of anorganic nitrogen in water
$DIN$ | $g\ kg^{-1}$ | $\frac{NS_{sminn}\ s\ f}{WS_{tot\_soil}} $ (linear?!) | dissolved anorganic nitrogen in water solution 
$Q_{dis}$ | $kg\ m^{-2}\ s^{-1}$ | - | hydrologic discharge from soil column to streamflow
$NF_{leached}$ | $g\ m^{-2} s^{-1}$ | $min(DIN\ Q_{dis}, \frac{NS_{sminn}\ s\ f}{\Delta t} )$ (linear?!) | leaching loss of nitrogen

Processes:
- Deposition of atmospheric nitrogen
- Biological nitrogen fixation
- Nitrification and denitrification
- Leaching of nitrogen

Bottlenecks:
- Steady state approximations (e.g. constants in BNF, maximum denitrification rate, nitrification flux)
- Binning of seperate processes (e.g. deposition of different atmospheric nitrogen compounds to one variable)
- Simplified fluxes (e.g. atmospheric N deposition and BNF contribute solely to NH$_4$ pool)
- Proxying processes by others (e.g. BNF by evapotranspiration)

Connected submodels:
- 2.18. Fixation and Uptake of Nitrogen
- 2.19. Carbon and Nitrogen allocation
- 2.24 Fire
...

Concluding comment:
*We should keep in mind that N and C cicle are highly dependent of each other.*

In [6]:
model = {}
model["key"] = "2.22"
model["model"] = "clm"
model["submodel"] = "n_cycle"
# model["date_summarized"] = arrow.now().format(fmt= 'YYYY-MM-DD HH:mm')
model["date_summarized"] = '2022-04-25 17:01'
model["link"] = "https://escomp.github.io/ctsm-docs/versions/master/html/tech_note/External_Nitrogen_Cycle/CLM50_Tech_Note_External_Nitrogen_Cycle.html#"
model["useful_references"] = ["kovenEffectVerticallyResolved2013", "thomasNitrogenLimitationLand2015"]

model["c1"] = (3, "Directly linked to biogeochemical flows")
model["c2"] = (2, "Nitrogen availability is an important factor for forests")
model["c3"] = (1, "Not directly linked to climate change issues")
model["c4"] = (2, "Many equations but still not too complex")
model["c5"] = (3, "Involves a lot of parameters, several linked exponentially")
model["c6"] = (3, "Well referenced and parameters nicely defined")
model["c7"] = (2, "Depends on several other nitrogen and water submodels, but builds a good independent core.")
model["c8"] = (1, "No direct connection to biotic evolution")


all_models_ranked.append(model)

<hr style="border-style: dashed;">

#### 5. Momentum, Sensible Heat, and Latent Heat Fluxes

This submodel deals with heat and vapor fluxes between atmosphere layers, vegetation and ground. It is highly complex.

Vocabulary:
- Zonal (latitudinal/east-west direction) and meridional (longitudinal/north-south direction) flows. 
- Momentum fluxes (a.k.a. winds)
- Sensible heat (a.k.a. temperature)
- Latent heat (heat energy that is exchanged to make phase trasitions, e.g. vaporization)

The model parameters are:


...too many

<div style="width:100px">parameter</div> | <div style="width:100px">unit</div>  | <div style="width:300px">equation (relation)</div> | <div style="width:500px">description</div>  |
|:---:|:---:|:---:|:---|
- | - | - | -

Processes:
- Atmospheric fluxes of momentum, vapor and sensible heat
- Fluxes of sensible and latent heat to and from ground
- Fluxes of sensible and latent heat to and from vegetated surfaces

Bottlenecks:
- Steady state approximations (e.g. dense canopy turbation)
- Discretization (e.g. processes by stabilities)
- Human classification (e.g. aerodynamic parameters for plant functional types, clear distinction between vegetated and non-vegetated surfaces)

Connected submodels:
- 2.6. Soil and snow temperatures
- 2.7. Hydrology
- 2.8. Snow hydrology
- 2.30. Dust model

Concluding comment: 
*Could be seperated into subsequent models; e.g. biotic and abiotic thermal fluxes.*

In [11]:
model = {}
model["key"] = "2.5"
model["model"] = "clm"
model["submodel"] = "momentum_heat_fluxes"
# model["date_summarized"] = arrow.now().format(fmt= 'YYYY-MM-DD HH:mm')
model["date_summarized"] = '2022-04-26 16:59'
model["link"] = "https://escomp.github.io/ctsm-docs/versions/master/html/tech_note/Fluxes/CLM50_Tech_Note_Fluxes.html#"
model["useful_references"] = []

model["c1"] = (3, "Thermic fluxes are key measures for predicting climate change.")
model["c2"] = (2, "Temperature changes define vital ranges of forests")
model["c3"] = (3, "Thermic fluxes are key measures for predicting climate change.")
model["c4"] = (3, "Very complex model")
model["c5"] = (3, "Involves a huge amount of parameters, several linked exponentially")
model["c6"] = (1, "Well referenced and parameters nicely defined, but the model seems overwhelming")
model["c7"] = (2, "Connected with many other temperature related submodels, but deals with distinct questions")
model["c8"] = (2, "Climate changes shift bitoic distributions")

all_models_ranked.append(model)

<hr style="border-style: dashed;">

#### 17. Plant respiration

Here authors deal with respiration of plants for maintainance and growth.

Vocabulary:
- plant functional type (PFT) as rough classification of vegetation type

The model parameters are:


<div style="width:100px">parameter</div> | <div style="width:100px">unit</div>  | <div style="width:400px">equation (relation)</div> | <div style="width:500px">description</div>  |
|:---:|:---:|:---:|:---|
$NS_{narea}$ | $g\ m^{-2}$ | - | leaf nitrogen content
$t_{2m,10days}$ | $°C$ | - | average temperature of last 10 days
$i_{atkin,pft}$ | $nmol\ g^{-1}\ s^{-1}$ | - | Atkin constant (intercept) which is PFT dependent (atkinGlobalVariabilityLeaf2015)
$CF_{mr\_leaf}$ | $nmol\ g^{-1}\ s^{-1}$ | $i_{atkin,pft} + (c_1\ NS_{narea}) - (c_2\ t_{2m,10days})$ (linear) | leaf respiration in light
$CF_{mr\_livestem}$ | $nmol\ g^{-1}\ s^{-1}$ | $NS_{livestem}\ MR_{base}\ MR_{Q10}^{(T_{2m}-20)/10}$ (linear?) | respiration of stem
$CF_{mr\_liveroot}$ | $nmol\ g^{-1}\ s^{-1}$ | $NS_{liveroot}\ MR_{base}\ MR_{Q10}^{(T_{2m}-20)/10}$ (linear?) | respiration of root
$CF_{mr\_froot}$ | $nmol\ g^{-1}\ s^{-1}$ | $\sum_{j=1}^{n_{levsoil}} NS_{froot}\ rootfr_j\ MR_{base}\ MR_{Q10}^{(T_{s_j}-20)/10}$ (linear?) | respiration of fine roots of different soil levels
$CF_{mr}$ | $nmol\ g^{-1}\ s^{-1}$ | $CF_{mr\_leaf} + CF_{mr\_froot} + CF_{mr\_livestem} + CF_{mr\_liveroot}$ (linear?) | respiration for maintainance
$CF_{growth}$ | $nmol\ g^{-1}\ s^{-1}$ | $0.11\ CF_{alloc?}$ (linear) | respiration for growth

Processes:
- Respiration for maintainance
- Respiration for growth

Bottlenecks:
- Steady state approximations (e.g. growth respiration)
- Discretization (e.g. fine roots on discrete soil levels)
- Problems to seperate processes (e.g. maintainance and growth respiration)

Connected submodels:
- 2.5. Momentum, sensible heat, and latent heat fluxes
- 2.9. Stomatal resistance and photosynthesis
- 2.16. CN pools
- 2.18. Fixation and uptake of nitrogen
- 2.19. Carbon and nitrogen allocation
...

Concluding comment:
*Canopy structure is not accounted for.*

In [5]:
model = {}
model["key"] = "2.17"
model["model"] = "clm"
model["submodel"] = "plant_respiration"
# model["date_summarized"] = arrow.now().format(fmt= 'YYYY-MM-DD HH:mm')
# print(model["date_summarized"])
model["date_summarized"] = '2022-04-27 12:11'
model["link"] = "https://escomp.github.io/ctsm-docs/versions/master/html/tech_note/Plant_Respiration/CLM50_Tech_Note_Plant_Respiration.html"
model["useful_references"] = ["atkinGlobalVariabilityLeaf2015", "atkinLeafRespirationTerrestrial2017"]

model["c1"] = (3, "Linked to climate change")
model["c2"] = (3, "Important to understand forest ecosystems")
model["c3"] = (3, "Respiration of plants and forests is key to understandd climate change.")
model["c4"] = (1, "The model (but not the connected ones) seems to be not very complex.")
model["c5"] = (2, "There are several PFT specific parameters defined, but moslty linearily connected.")
model["c6"] = (3, "Due to its small size and the good documentation it seems to be easily reimplemented.")
model["c7"] = (1, "Stongly connected to carbon and nitrogen pools and allocation, as well as soil mositure and heat fluxes.")
model["c8"] = (3, "Plant respiration is an adaptive trait.")


all_models_ranked.append(model)

<hr style="border-style: dashed;">

#### 21. Decomposition

This submodel focusses on the decay of organic material to more inert recalcitrant organic matter. It assumes cascades of decomposition.

Vocabulary:
- recalcitrant organic matter is resistent to decomposition

The model parameters are:


<div style="width:100px">parameter</div> | <div style="width:100px">unit</div>  | <div style="width:300px">equation (relation)</div> | <div style="width:500px">description</div>  |
|:---:|:---:|:---:|:---|
$R_i$ | - | - | direct carbon input by plants to soil level
$k_i$ | - | - | decomposition rate constant of soil level
$C_i$ | - | - | carbon content of soil level
$T_i$ | - | - | decomposition rate constant of specific level
$r_i$ | - | - | fraction of carbon loss to respiration of soil level
$\frac{\partial C_i}{\partial t}$ | $g\ m^{-3}$ | $R_i + \sum_{j \not i}(i-r_j)\ T_{ji}\ k_j\ C_j - k_i\ C_i$ | level specific carbon balance of decomposing pools
$r_{total}$ | - | $r_{tsoil}\ r_{water}\ r_{oxygen}\ r_{depth}$ | environmental decomposition factor of temperature, moisture, anoxia and layer depth
$CS_u$ | $g\ m^{-2}$ | - | initial mass in upstream pool
$CF_{pot,u}$ | $g\ m^{-2}\ s^{-1}$ | $CS_u\ k_u$ (linear?)| potential carbon flux of upstream pool
$NF_{pot\_min,u\to d}$ | $g\ m^{-2}\ s^{-1}$ | $\frac{CF_{pot,u}(1-r\ f_u - \frac{CN_d}{CN_u})}{CN_d}$ | potential mineral nitrogen fluxes from upstream to downstream layer
... | - | - | ...

Processes:
- decomposition of organic matter in soil levels
- fluxes of organic matter between soil levels
- environmental modification of decomposition rates
- heterotrophic respiration to atmosphere
- nitrogen immobilization and mineralization
- nitrogen competition between decomposition and plants

Bottlenecks:
- Binning of seperate processes (e.g. environmental limitation to decay by depth as bin for multiple processes)
- Discretization (e.g. discrete classes of soil organic matter and litter)
- Steady state approximations (e.g. decomposition rate constants)

Connected submodels:
- 2.2. Surface characterization, vertical discretization, and model input requirements
- 2.6. Soil and snow temperatures
- 2.16. CN pools
- 2.22. Fixation and uptake of nitrogen (FUN)
...

Concluding comment:
*Very explicit cascade model, with many assumptions.*

In [9]:
model = {}
model["key"] = "2.21"
model["model"] = "clm"
model["submodel"] = "decomposition"
model["date_summarized"] = '2022-04-27 16:13'
model["link"] = "https://escomp.github.io/ctsm-docs/versions/master/html/tech_note/Decomposition/CLM50_Tech_Note_Decomposition.html"
model["useful_references"] = ["kovenEffectVerticallyResolved2013"]

model["c1"] = (3, "Decompostition is part of biogeochemical processes, thus delas with planetary boundary.")
model["c2"] = (2, "Important processes in forests")
model["c3"] = (2, "Decomposition is climate dependent and contributes to green hose gases.")
model["c4"] = (3, "There is one hint about the performance of the model in 'Because of the coupling between the slowest SOM pools and productivity through N downregulation of photosynthesis, equilibration of the model for initialization purposes will take an extremely long time in the standard model.'.")
model["c5"] = (3, "As all layers are dependent of each other and multiple parameters are involved, I assume a strong impact of varying parameters.")
model["c6"] = (2, "Well documented but seems very big to reimplement, with all given consitionalities")
model["c7"] = (2, "Deals with subsequent processes, but depends on several other submodels")
model["c8"] = (3, "Decomposition as biological process is performed by microbial communities; clear evolutionary context")


all_models_ranked.append(model)

<hr style="border-style: dashed;">

#### 25. Methane Model

Here, the model deals with methane production and its fluxes within bio- and atmosphere.

The model parameters are:


<div style="width:100px">parameter</div> | <div style="width:100px">unit</div>  | <div style="width:300px">equation (relation)</div> | <div style="width:500px">description</div>  |
|:---:|:---:|:---:|:---|
$NF_{ndep\_sminn}$ | $g\ m^{-2}\ s^{-1}$ | - | total deposition of mineral nitrogen to land surface (NO$_{x}$, NH$_{y}$)


Processes:
-


Bottlenecks:
- 

Connected submodels:
- 


Concluding comment:


In [7]:
model = {}
model["key"] = "cc"
model["model"] = "clm"
model["submodel"] = "cccc"
model["date_summarized"] = arrow.now().format(fmt= 'YYYY-MM-DD HH:mm')
print(model["date_summarized"])
# model["date_summarized"] = 'cc'
model["link"] = "c"
model["useful_references"] = ["ddd"]

model["c1"] = (3, "Direct connection with biogeochemical fluxes and climate change (as green house gas)")
model["c2"] = (2, "scds")
model["c3"] = (3, "Direct connection with climate change (as green house gas)")
model["c4"] = (2, "sdvdssv")
model["c5"] = (3, "vsvsv")
model["c6"] = (3, "sbsbs")
model["c7"] = (2, "ter")
model["c8"] = (1, "gbe")


# all_models_ranked.append(model)

<hr style="border-style: dashed;">

#### xxx. copy

...

The model parameters are:


<div style="width:100px">parameter</div> | <div style="width:100px">unit</div>  | <div style="width:300px">equation (relation)</div> | <div style="width:500px">description</div>  |
|:---:|:---:|:---:|:---|
$NF_{ndep\_sminn}$ | $g\ m^{-2}\ s^{-1}$ | - | total deposition of mineral nitrogen to land surface (NO$_{x}$, NH$_{y}$)


Processes:
-


Bottlenecks:
- 

Connected submodels:
- 


Concluding comment:


In [7]:
model = {}
model["key"] = "cc"
model["model"] = "clm"
model["submodel"] = "cccc"
model["date_summarized"] = arrow.now().format(fmt= 'YYYY-MM-DD HH:mm')
print(model["date_summarized"])
# model["date_summarized"] = 'cc'
model["link"] = "c"
model["useful_references"] = ["ddd"]

model["c1"] = (3, "scs")
model["c2"] = (2, "scds")
model["c3"] = (1, "sc")
model["c4"] = (2, "sdvdssv")
model["c5"] = (3, "vsvsv")
model["c6"] = (3, "sbsbs")
model["c7"] = (2, "ter")
model["c8"] = (1, "gbe")


# all_models_ranked.append(model)